# How To Compute Receptive Visual Fields
This notebook was created as a skeleton to compute receptive visual fields from data collected with neuropixels probes. This notebook is based on one created by Dan as an assignment for his NRSC7610 course and will almost certainly require adjustment based on changes in data acquisition and variation in projects. One major change will likely result from the use of *Kilosort2* vs *Spyking-Circus* which was used in the original notebook. The stimuli and region of measurement will also vary from project to project. **DO NOT use this skeleton as written. This is only a framework for analysis of future projects**

## Spike Sorting with Kilosort2
Kilosort2 is a MatLab package for spike sorting ephys data up to 1024 channels. Automated kilosort output requires little curation for Neuropixels.  
Recorded voltage deflection waveforms have characteristic spatial shapes determined by neuron's location and physiological characteristics. These characteristics can be used to sort inividual spiking neurons. Traditional spike sorting method for smaller arrays can take days to run. With high density probes, waveforms of each neuron can be recorded on 5-50 channels simultaneously. Clustering algorithm required to unmix signals and assign spikes. 

## Package Import

In [ ]:
import numpy as np
import os,sys,glob, h5py
import matplotlib.pyplot as plt
from utils import cleanAxes

## Load Data

In [ ]:
nwb_data = h5py.File()

The data is in an HDF5 (Hierarchical Data Format). Similar in concept to a nested Python `dictionary` with data grouped into sections by 'keys'. (Assuming Similarity to NRSC7610) The most important keys will be `processing` and `stimulus`. Processing includes the results of spike sorting. Raw data "processed" into spike times which are assigned to single neurons

## Get Spike Times

View keys within nwb_data

In [ ]:
nwb_data.keys()

**Get spike times for an individual neuron**  
Select a neuron and replace `*NUMBER*` with the neuron number. The second cell below is a stand-in the final iteration needed to get spike times. You will need to use `.keys()` to know exactly which keys will lead you to spike times 

In [ ]:
#Repeat until you're ready for the block below
nwb_data['...']['...'].keys()

In [ ]:
spike_times = nwb_data['processing']['...']['UnitTimes']['*NUMBER*']['times']

Convert to a numpy array

In [ ]:
spike_times = np.array(spike_times)

## Isolate Stimulus Data

Along with the spike data, `nwb_data` also contains data regarding our stimuli. Using the same method as above, we're going to find specific stimulus data. 

In [ ]:
#Repeat until you've found the stimuli of interest
nwb_data.keys()
nwb_data['...'].keys

Below is an example from a prior project of where you should end up more or less. Replace the keys in brackets as necessary. The keys we're hoping for here are `data` and `timestamps` or something similar

In [ ]:
nwb_data['stimulus']['presentation']['*binary_green*'].keys()

`timestamps` is a 1D array containing the time of each frame. `data` is a 3D array with each frame (64x64) matching one of the timestamps. We will assign the variable `stimulus` to `data`(The actual stimuli on each frame) and assign `stimulus times` to `timestamps`. **Make sure to replace `*Stimulus Type*` with the actual stimulus type youre workign with as well as any other keys that are different based on your work above**

In [ ]:
stimulus = np.array(nwb_data['stimulus']['presentation']['*STIMULUS TYPE*']['data']).T
stimulus_times = np.array(nwb_data['stimulus']['presentation']['*STIMULUS TYPE*']['timestamps'])+0.04 # this an adjustment for the hardware used, can ignore

## Calculating Receptive Fields

Now we're able to calculate the receptive field of our individual neuron. For this, we're going to use the *spike-triggered average method*. For this, we're going to make an average of the stimulus frames (i.e., `data`, 64x64), but **only those immediately preceding a spike**. For this we first need to know the `timestamps` that immediately precede a spike.

In [ ]:
print('Spike times go from '+str(spike_times[0])+' to '+str(spike_times[-1])+' seconds')
print('Stimulus times go from '+str(stimulus_times[0])+' to '+str(stimulus_times[-1])+' seconds')

This gives the range of spike times where there is stimulus and vice versa. You're now going to use these times with `np.where()` to select the spike times that occurred during the stimulus. These times will go into a numpy array labeled `stimulus_spike_times`

In [ ]:
indexes = np.where([spike_times> *TIME* , spike_times< *TIME* ])
stimulus_spike_times = np.array(spike_times[indexes[1]])
print(stimulus_spike_times)

Now we're going to iterate over these and find the frames presented before each spike. First we need to set a standard time for signal transduction to the region being recorded from. (e.g. Retina to LGN = 90ms). As well, we should calculate `number_of_frames` for the upcoming step

In [ ]:
time_before_spike = X.XXX #in seconds
number_of_frames = len(nwb_data['stimulus']['presentation']['binary_uv']['timestamps'])
#Replace keys as needed

Now we need to assign a frame number to each spike time (index in the range of `0` to `number_of_frames`) that occured `time_before_spike` before each spike. Then make an array of these indices labeled `stimulus_frame_indices`

In [ ]:
stimulus_frame_indices = np.array([])
for x in stimulus_spike_times:
    adjusted_x = x - time_before_spike
    stimulus_frames = np.where(stimulus_times > adjusted_x)
    stimulus_frame_index = (np.amin(np.array(stimulus_frames)))-1
    stimulus_frame_indices = np.append(stimulus_frame_indices, [stimulus_frame_index], axis = 0)
print(stimulus_frame_indices)
stimulus_frame_indices.shape

In [ ]:
spatial_receptive_fields = np.zeros([64,64])
for i in stimulus_frame_indices:
    i = int(i)
    spatial_receptive_fields = stimulus[:,:,i] + spatial_receptive_fields
spatial_receptive_fields = spatial_receptive_fields / len(stimulus_frame_indices)

## Plot the Receptive Field

In [ ]:
plt.imshow(spatial_receptive_fields,clim=(-0.2,0.2),cmap=plt.cm.Greys)